## Live holdings from orders

This notebook shows how you can get an order-adjusted view into your holdings using the [GetHoldingsWithOrders](https://www.lusid.com/docs/api/#operation/GetHoldingsWithOrders) endpoint. For the purposes of this demo, we'll generate two IBM orders in a portfolio, across two strategies of **Tech** and **Balanced**. We then show:

* Holdings with unbooked orders
* Holdings with partially booked orders

In [1]:
#%load_ext nb_black

### Setup LUSID

In [2]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import inspect
import os

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH3")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
API Version:  0.6.4645.0


In [3]:
orders_scope = "orders"
#orders_scope = create_scope_id().replace("-","")
orders_portfolio = "us_long_live"
strategy_shk = "strategy"
executed_price_code = "executed_price"
execution_price_property = f"Allocation/{orders_scope}/{executed_price_code}"
order_strategy_property = f"Transaction/{orders_scope}/{strategy_shk}"
allocation_strategy_property = f"Allocation/{orders_scope}/{strategy_shk}"
pf_created_date = "2020-01-01T00:00:00+00:00"

### Load an IBM equity instrument into LUSID

In [4]:
instruments_df = pd.read_csv("data/instruments.csv")
display(instruments_df)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,IBM,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,IBM,equity,common_stock,NaN


In [5]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {"name": "instrument_name"},
}

In [6]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=instruments_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,1,0,0


### Create properties for the order and allocations

In [7]:
properties_api = api_factory.build(la.PropertyDefinitionsApi)


def create_property_definition(domain, scope, code, data_type):
    try:
        properties_api.create_property_definition(
            lm.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time="Perpetual",
                value_required=False,
                data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type),
            )
        )
    except ApiException as e:
        detail = json.loads(e.body)
        print(detail["title"])
        if detail["code"] != 124:  # 'PropertyAlreadyExists'
            raise e

In [8]:
# sub-holding keys
create_property_definition("Transaction", orders_scope, strategy_shk, "string")

# allocation execution price
create_property_definition("Allocation", orders_scope, executed_price_code, "number")

Error creating Property Definition 'Transaction/orders/strategy' because it already exists.
Error creating Property Definition 'Allocation/orders/executed_price' because it already exists.


### Create a portfolio

In [9]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(
        data=[{"portfolio_code": portfolio_code, "portfolio_name": name},]
    )

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {"created": f"${pf_created_date}"},
    }

    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[strategy_shk],
    )

    succ, failed = format_portfolios_response(result)
    display(
        pd.DataFrame(
            data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
        )
    )
    print(list(failed["error_items"]))

In [10]:
create_portfolio(orders_scope, orders_portfolio, "ibor")

,success,failed,errors
0,1,0,0


[]


### Set holdings in portfolio

In [11]:
hldgs_df = pd.read_csv("data/initial_holdings.csv")
display(hldgs_df)

,instrument_name,ccyId,client_internal,isin,figi,quantity,unit_cost,total_cost,currency,strategy
0,IBM,NaN,imd_34634534,US0231351067,BBG000BVPXP1,1000,150,150000,USD,Tech
1,IBM,NaN,imd_34634534,US0231351067,BBG000BVPXP1,200,150,30000,USD,Balanced
2,USD,USD,NaN,NaN,NaN,1000,1,1000,USD,Tech
3,USD,USD,NaN,NaN,NaN,1000,1,1000,USD,Balanced


In [12]:
holdings_mapping = {
    "required": {
        "code": f"${orders_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity",
    },
    "identifier_mapping": {"Figi": "figi", "Currency": "ccyId"},
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost",
    },
}

In [13]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    sub_holding_keys=[strategy_shk],
    file_type="holdings",
)

succ, failed = format_holdings_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,1,0,0


In [14]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings(
    scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD
1,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD
2,CCY_USD,Tech,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD
3,CCY_USD,Balanced,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD


### Raise some orders

Here we raise two orders on the same IBM instrument:

* One BUY order for 100 units on the Tech strategy
* One SELL order for 75 units on the Balanced startegy

In [15]:
orders_df = pd.read_csv("data/orders.csv")
orders_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,strategy
0,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,100,0.9,USD,ORD001,buy limit,Tech
1,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,75,1.1,USD,ORD002,sell limit,Balanced


In [16]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():

    portfolio = order["portfolio"]

    order_requests[portfolio].append(
        models.OrderRequest(
            id=models.ResourceId(scope=orders_scope, code=order["order_id"]),
            quantity=order["quantity"],
            side=order["side"],
            instrument_identifiers={
                "Instrument/default/Figi": order["figi"],
                "Instrument/default/Name": order["instrument_name"],
            },
            properties={
                "Order/system/Currency": models.PerpetualProperty(
                    key="Order/system/Currency",
                    value=models.PropertyValue(order["currency"]),
                ),
                "Order/system/LimitPrice": models.PerpetualProperty(
                    key="Order/system/LimitPrice",
                    value=models.PropertyValue(order["price"]),
                ),
                "Order/system/State": models.PerpetualProperty(
                    key="Order/system/State", value=models.PropertyValue("Open")
                ),
                f"Transaction/{orders_scope}/{strategy_shk}": models.PerpetualProperty(
                    key=f"Transaction/{orders_scope}/{strategy_shk}",
                    value=models.PropertyValue(order["strategy"]),
                ),
            },
            order_book_id=models.ResourceId(scope=orders_scope, code=orders_portfolio,),
            portfolio_id=models.ResourceId(scope=orders_scope, code=order["portfolio"]),
        )
    )

for order_portfolio in order_requests:
    order_sets[order_portfolio] = models.OrderSetRequest(
        order_requests=order_requests[order_portfolio]
    )

for order_portfolio in order_sets:
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=order_sets[order_portfolio]
    )

    display(lusid_response_to_data_frame(response, rename_properties=True))

,Currency(system-Properties),LimitPrice(system-Properties),State(system-Properties),strategy(orders-Properties),version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id
0,USD,1.1,Open,Balanced,0001-01-01 00:00:00+00:00,2020-06-19 12:31:12.322793+00:00,BBG000BVPXP1,IBM,75,sell limit,orders,us_long_live,orders,us_long_live,orders,ORD002,LUID_RXNFVIGN
1,USD,0.9,Open,Tech,0001-01-01 00:00:00+00:00,2020-06-19 12:31:12.322793+00:00,BBG000BVPXP1,IBM,100,buy limit,orders,us_long_live,orders,us_long_live,orders,ORD001,LUID_RXNFVIGN


### ... and show holdings with orders

Here we can see two orders on the same IBM instrument:

* One BUY order for 100 units on the Tech strategy (with a cost of -90 USD @ price of 1.1)
* One SELL order for 75 units on the Balanced startegy (with a cost of + 82.5 USD @ price of 0.9)

In [17]:
executed_holdings = api_factory.build(
    la.TransactionPortfoliosApi
).get_holdings_with_orders(
    scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_identifiers.Instrument/default/Name,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.unit,transaction.source
0,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,O,-75.0,0.0,82.5,USD,82.5,USD,VirtualTrade-Order-orders/ORD002-4c467,Sell,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:15.003467+00:00,2020-06-22 12:31:15.003467+00:00,-75.0,1.1,Price,82.5,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
3,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,O,100.0,0.0,-90.0,USD,-90.0,USD,VirtualTrade-Order-orders/ORD001-3c2ec,Buy,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:15.003467+00:00,2020-06-22 12:31:15.003467+00:00,100.0,0.9,Price,-90.0,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
4,CCY_USD,Tech,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CCY_USD,Balanced,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CCY_USD,Balanced,CCY_USD,us_long_live,orders,OC,82.5,0.0,-82.5,USD,-82.5,USD,VirtualTrade-Order-orders/ORD002-4c467,Sell,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:15.003467+00:00,2020-06-22 12:31:15.003467+00:00,-75.0,1.1,Price,82.5,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
7,CCY_USD,Tech,CCY_USD,us_long_live,orders,OC,-90.0,0.0,90.0,USD,90.0,USD,VirtualTrade-Order-orders/ORD001-3c2ec,Buy,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:15.003467+00:00,2020-06-22 12:31:15.003467+00:00,100.0,0.9,Price,-90.0,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,


In [18]:
orders_api = api_factory.build(lusid.api.OrdersApi)

### ... and publish some allocations

In the cells below, we partially fill the allocations:

* We fill 82 of 100 for the Tech strategy
* We fill 61 of -75 in the Balanced strategy

In [19]:
allocations_df = pd.read_csv('data/allocations.csv')
allocations_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order,strategy,side
0,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,82,0.9,USD,ALLOC001,ORD001,Tech,buy limit
1,us_long_live,IBM,imd_34634534,US0231351067,BBG000BVPXP1,61,1.1,USD,ALLOC002,ORD002,Balanced,sell limit


In [20]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():

    portfolio = allocation["portfolio"]

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(scope=orders_scope, code=allocation["allocation_id"]),
            allocated_order_id=models.ResourceId(
                scope=orders_scope, code=allocation["originating_order"]
            ),
            quantity=allocation["quantity"],
            instrument_identifiers={
                "Instrument/default/Figi": allocation["figi"],
                "Instrument/default/Name": allocation["instrument_name"],
            },
            properties={
                "Allocation/system/Currency": models.PerpetualProperty(
                    key="Allocation/system/Currency",
                    value=models.PropertyValue(allocation["currency"]),
                ),
                "Allocation/system/LimitPrice": models.PerpetualProperty(
                    key="Allocation/system/LimitPrice",
                    value=models.PropertyValue(allocation["price"]),
                ),
                "Allocation/system/Side": models.PerpetualProperty(
                    key="Allocation/system/Side",
                    value=models.PropertyValue(allocation["side"]),
                ),
                "Allocation/system/State": models.PerpetualProperty(
                    key="Allocation/system/State", value=models.PropertyValue("Open")
                ),
                f"Transaction/{orders_scope}/{strategy_shk}": models.PerpetualProperty(
                    key=f"Transaction/{orders_scope}/{strategy_shk}",
                    value=models.PropertyValue(allocation["strategy"]),
                ),
            },
            portfolio_id=models.ResourceId(
                scope=orders_scope, code=allocation["portfolio"]
            ),
        )
    )

for allocation_portfolio in allocation_requests:
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )

for allocation_portfolio in allocation_sets:
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=allocation_sets[allocation_portfolio]
    )

    display(lusid_response_to_data_frame(response, rename_properties=True))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,version.effective_from,version.as_at_date,Currency(system-Properties),LimitPrice(system-Properties),Side(system-Properties),State(system-Properties),strategy(orders-Properties),lusid_instrument_id
0,orders,ALLOC001,orders,ORD001,orders,us_long_live,82,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-19 12:31:17.558469+00:00,USD,0.9,buy limit,Open,Tech,LUID_RXNFVIGN
1,orders,ALLOC002,orders,ORD002,orders,us_long_live,61,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-19 12:31:17.558469+00:00,USD,1.1,sell limit,Open,Balanced,LUID_RXNFVIGN


### ... and look at holdings again

In [21]:
executed_holdings = api_factory.build(
    la.TransactionPortfoliosApi
).get_holdings_with_orders(
    scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_identifiers.Instrument/default/Name,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.unit,transaction.source
0,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,P,1000.0,1000.0,150000.0,USD,150000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,P,200.0,200.0,30000.0,USD,30000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,O,-14.0,0.0,15.4,USD,15.4,USD,VirtualTrade-Order-orders/ORD002-df438,Sell,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:17.744657+00:00,2020-06-22 12:31:17.744657+00:00,-14.0,1.1,Price,15.4,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
3,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,O,18.0,0.0,-16.2,USD,-16.2,USD,VirtualTrade-Order-orders/ORD001-1d106,Buy,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:17.744657+00:00,2020-06-22 12:31:17.744657+00:00,18.0,0.9,Price,-16.2,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
4,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,L,-61.0,0.0,67.1,USD,67.1,USD,VirtualTrade-Order-orders/ALLOC002-1f8b5,Sell,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:17.744657+00:00,2020-06-22 12:31:17.744657+00:00,-61.0,1.1,Price,67.1,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
5,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,L,82.0,0.0,-73.8,USD,-73.8,USD,VirtualTrade-Order-orders/ALLOC001-207de,Buy,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:17.744657+00:00,2020-06-22 12:31:17.744657+00:00,82.0,0.9,Price,-73.8,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
6,CCY_USD,Tech,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CCY_USD,Balanced,CCY_USD,us_long_live,orders,B,1000.0,1000.0,1000.0,USD,1000.0,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CCY_USD,Balanced,CCY_USD,us_long_live,orders,OC,15.4,0.0,-15.4,USD,-15.4,USD,VirtualTrade-Order-orders/ORD002-df438,Sell,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:17.744657+00:00,2020-06-22 12:31:17.744657+00:00,-14.0,1.1,Price,15.4,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,
9,CCY_USD,Tech,CCY_USD,us_long_live,orders,OC,-16.2,0.0,16.2,USD,16.2,USD,VirtualTrade-Order-orders/ORD001-1d106,Buy,BBG000BVPXP1,IBM,LUID_RXNFVIGN,2020-06-19 12:31:17.744657+00:00,2020-06-22 12:31:17.744657+00:00,18.0,0.9,Price,-16.2,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,


### Now close the orders and allocations

In [22]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():

    portfolio = order["portfolio"]

    order_requests[portfolio].append(
        models.OrderRequest(
            id=models.ResourceId(scope=orders_scope, code=order["order_id"]),
            quantity=order["quantity"],
            side=order["side"],
            instrument_identifiers={
                "Instrument/default/Figi": order["figi"],
                "Instrument/default/Name": order["instrument_name"],
            },
            properties={
                "Order/system/Currency": models.PerpetualProperty(
                    key="Order/system/Currency",
                    value=models.PropertyValue(order["currency"]),
                ),
                "Order/system/LimitPrice": models.PerpetualProperty(
                    key="Order/system/LimitPrice",
                    value=models.PropertyValue(order["price"]),
                ),
                "Order/system/State": models.PerpetualProperty(
                    key="Order/system/State", value=models.PropertyValue("Closed")
                ),
                f"Transaction/{orders_scope}/{strategy_shk}": models.PerpetualProperty(
                    key=f"Transaction/{orders_scope}/{strategy_shk}",
                    value=models.PropertyValue(order["strategy"]),
                ),
            },
            order_book_id=models.ResourceId(scope=orders_scope, code=orders_portfolio,),
            portfolio_id=models.ResourceId(scope=orders_scope, code=order["portfolio"]),
        )
    )

for order_portfolio in order_requests:
    order_sets[order_portfolio] = models.OrderSetRequest(
        order_requests=order_requests[order_portfolio]
    )

for order_portfolio in order_sets:
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=order_sets[order_portfolio]
    )

    display(lusid_response_to_data_frame(response, rename_properties=True))

allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():

    portfolio = allocation["portfolio"]

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(scope=orders_scope, code=allocation["allocation_id"]),
            allocated_order_id=models.ResourceId(
                scope=orders_scope, code=allocation["originating_order"]
            ),
            quantity=allocation["quantity"],
            instrument_identifiers={
                "Instrument/default/Figi": allocation["figi"],
                "Instrument/default/Name": allocation["instrument_name"],
            },
            properties={
                "Allocation/system/Currency": models.PerpetualProperty(
                    key="Allocation/system/Currency",
                    value=models.PropertyValue(allocation["currency"]),
                ),
                "Allocation/system/LimitPrice": models.PerpetualProperty(
                    key="Allocation/system/LimitPrice",
                    value=models.PropertyValue(allocation["price"]),
                ),
                "Allocation/system/Side": models.PerpetualProperty(
                    key="Allocation/system/Side",
                    value=models.PropertyValue(allocation["side"]),
                ),
                "Allocation/system/State": models.PerpetualProperty(
                    key="Allocation/system/State", value=models.PropertyValue("Closed")
                ),
                f"Transaction/{orders_scope}/{strategy_shk}": models.PerpetualProperty(
                    key=f"Transaction/{orders_scope}/{strategy_shk}",
                    value=models.PropertyValue(allocation["strategy"]),
                ),
            },
            portfolio_id=models.ResourceId(
                scope=orders_scope, code=allocation["portfolio"]
            ),
        )
    )

for allocation_portfolio in allocation_requests:
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )

for allocation_portfolio in allocation_sets:
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=allocation_sets[allocation_portfolio]
    )

    display(lusid_response_to_data_frame(response, rename_properties=True))

,Currency(system-Properties),LimitPrice(system-Properties),State(system-Properties),strategy(orders-Properties),version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id
0,USD,1.1,Closed,Balanced,0001-01-01 00:00:00+00:00,2020-06-19 12:31:18.435627+00:00,BBG000BVPXP1,IBM,75,sell limit,orders,us_long_live,orders,us_long_live,orders,ORD002,LUID_RXNFVIGN
1,USD,0.9,Closed,Tech,0001-01-01 00:00:00+00:00,2020-06-19 12:31:18.435627+00:00,BBG000BVPXP1,IBM,100,buy limit,orders,us_long_live,orders,us_long_live,orders,ORD001,LUID_RXNFVIGN


,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,instrument_identifiers.Instrument/default/Name,version.effective_from,version.as_at_date,Currency(system-Properties),LimitPrice(system-Properties),Side(system-Properties),State(system-Properties),strategy(orders-Properties),lusid_instrument_id
0,orders,ALLOC001,orders,ORD001,orders,us_long_live,82,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-19 12:31:18.673535+00:00,USD,0.9,buy limit,Closed,Tech,LUID_RXNFVIGN
1,orders,ALLOC002,orders,ORD002,orders,us_long_live,61,BBG000BVPXP1,IBM,0001-01-01 00:00:00+00:00,2020-06-19 12:31:18.673535+00:00,USD,1.1,sell limit,Closed,Balanced,LUID_RXNFVIGN


### Book transactions from the allocations

In [23]:
transaction_mapping = {
        "identifier_mapping": {"LusidInstrumentId": "luid"},
        "required": {
            "code": "fund_code",
            "transaction_id": "trd_number",
            "type": "transaction_code",
            "transaction_price.price": "price",
            "transaction_price.type": "$Price",
            "total_consideration.amount": "principal",
            "units": "units",
            "transaction_date": "$2020-06-19",
            "total_consideration.currency": "trading_currency",
            "settlement_date": "$2020-06-19",
        },
        "optional": {},
        "properties": ["strategy"],
    }

In [24]:
trades_df = pd.DataFrame(
    
    { "fund_code": ["us_long_live", "us_long_live"],
    "trd_number": ["trd_ord_001", "trd_ord_002"],
    "transaction_code": ["Sell", "Buy"] ,
    "price": [1.1, 0.9],
    "principal": [67.1, 73.8] ,
    "units": [61, 82],
    "trading_currency": ["USD", "USD"],
    "luid": ["LUID_RXNFVIGN", "LUID_RXNFVIGN"],
    "strategy": ["Balanced", "Tech"]}
)

In [25]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=trades_df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=orders_scope
)

succ, failed = format_transactions_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


### And look at holdings for the final time

In [26]:
executed_holdings = api_factory.build(
    la.TransactionPortfoliosApi
).get_holdings_with_orders(
    scope=orders_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(executed_holdings, rename_properties=True)

,instrument_uid,strategy(orders-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_RXNFVIGN,Tech,IBM,us_long_live,orders,P,1082.0,1082.0,150073.8,USD,150000.0,USD
1,LUID_RXNFVIGN,Balanced,IBM,us_long_live,orders,P,139.0,139.0,20850.0,USD,20850.0,USD
2,CCY_USD,Tech,CCY_USD,us_long_live,orders,B,926.2,926.2,926.2,USD,926.2,USD
3,CCY_USD,Balanced,CCY_USD,us_long_live,orders,B,1067.1,1067.1,1067.1,USD,1000.0,USD


### Cancel the transactions

In [27]:
txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

cancel_response = txn_port_api.cancel_transactions(
    scope=orders_scope, code=orders_portfolio, transaction_ids=["trd_ord_001", "trd_ord_002"]
)